In [1]:
import os
# os.environ['DEVICE_ID'] = '0'
import numpy as np
import pandas as pd 
import mindspore as ms
from mindspore import nn,Tensor
from mindspore import context
from mindspore.nn.metrics import Accuracy, MAE, MSE

In [2]:
input_data = pd.read_csv('./data_train_input/data_train_input.csv')
output_data = pd.read_csv('./data_train_output/data_train_output.csv')
data = pd.concat([input_data,output_data],axis=1)
print('NAN %')
print((data.isna().sum()/data.shape[0]).apply(lambda x:format(x,'.2%')))
data = data.drop(['U_z','V_z','W_Z'],axis=1)
train = data.sample(frac=0.8,random_state=0)
test = data.drop(train.index)
x_train = train.iloc[:,:-6]
y_train = train.iloc[:,-6:]
x_test = test.iloc[:,:-6]
y_test = test.iloc[:,-6:]
x_train_ts = Tensor(np.array(x_train),ms.float32)
y_train_ts = Tensor(np.array(y_train),ms.float32)
x_test_ts = Tensor(np.array(x_test),ms.float32)
y_test_ts = Tensor(np.array(y_test),ms.float32)

NAN %
U       0.00%
V       0.00%
W       0.00%
U_x     0.00%
U_y     0.00%
U_z    52.83%
V_x     0.00%
V_y     0.00%
V_z    52.83%
W_x     0.00%
W_y     0.00%
W_Z    52.83%
UU      0.00%
UV      0.00%
UW      0.00%
VV      0.00%
VW      0.00%
WW      0.00%
dtype: object


In [180]:
import mindspore.dataset as ds
dataset = {'feature':x_train_ts[:10],'label':y_train_ts[:10] }
dataset = ds.NumpySlicesDataset(dataset, shuffle=False)
# shuffled_dataset = dataset.shuffle(buffer_size=1000)

In [181]:
print(dataset

In [96]:
x_train_ts

Tensor(shape=[1868997, 9], dtype=Float32, value=
[[ 1.17781997e+00,  1.16190001e-01,  0.00000000e+00 ... -1.85819507e-01,  0.00000000e+00,  0.00000000e+00],
 [ 2.72589713e-01, -2.00515445e-02, -6.06241217e-03 ... -7.44985640e-02,  5.93658304e-03, -4.30153077e-03],
 [ 1.02515376e+00, -3.69837359e-02,  5.96646499e-03 ...  3.52185369e-02, -1.95971719e-04, -5.05584059e-03],
 ...
 [ 1.08763897e+00, -4.89311107e-02,  0.00000000e+00 ...  6.31596670e-02,  0.00000000e+00,  0.00000000e+00],
 [ 7.24852324e-01,  8.17539403e-04,  0.00000000e+00 ... -6.14322210e-03,  0.00000000e+00,  0.00000000e+00],
 [ 3.38216603e-01, -1.16487248e-02, -3.33991344e-03 ... -4.03530449e-02,  4.27061412e-03,  2.49097142e-02]])

In [97]:
y_train_ts

Tensor(shape=[1868997, 6], dtype=Float32, value=
[[ 7.52748398e-04,  1.10478402e-04,  0.00000000e+00,  6.30408875e-04,  0.00000000e+00,  6.06322777e-04],
 [ 3.75780500e-02, -9.07959417e-03,  1.08092360e-03,  2.29724422e-02, -1.66198424e-05,  3.18249501e-02],
 [ 4.23792526e-02, -9.03332140e-03, -6.90772067e-05,  1.69404075e-02,  1.09453096e-04,  1.84522085e-02],
 ...
 [ 1.05965801e-03,  1.93561194e-04,  0.00000000e+00,  5.99977211e-04,  0.00000000e+00,  6.99713477e-04],
 [ 9.83286276e-03,  2.90450011e-03,  0.00000000e+00,  4.25822381e-03,  0.00000000e+00,  6.12206990e-03],
 [ 3.67839336e-02, -9.86265857e-03,  1.23296503e-03,  2.26253327e-02,  1.72280517e-04,  2.93803141e-02]])

In [ ]:
import numpy as np
import mindspore as ms
import mindspore.nn as nn
from mindspore import Tensor, Model
from mindspore.train.callback import LossMonitor
from mindspore import context
from mindspore.dataset import Dataset
from mindspore.common.initializer import Normal
from tqdm import tqdm

# Set the context to use CPU or GPU if available
context.set_context(mode=context.GRAPH_MODE, device_target="CPU")

# Generate some random data for demonstration purposes
num_samples = 100000
num_features = 9
num_labels = 6

# Generate random tabular data and labels
data = x_train_ts 
labels = y_train_ts 

# Define the regression model
class RegressionModel(nn.Cell):
    def __init__(self):
        super(RegressionModel, self).__init__()
        self.fc1 = nn.Dense(num_features, 64, weight_init=Normal(0.02))
        self.fc2 = nn.Dense(64, 32, weight_init=Normal(0.02))
        self.fc3 = nn.Dense(32, num_labels, weight_init=Normal(0.02))
        self.relu = nn.ReLU()
        self.flatten = nn.Flatten()

    def construct(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x

# Create a custom dataset
def generator():
    for i in range(num_samples):
        yield data[i], labels[i]

# Create the dataset and dataloader
dataset = ms.dataset.GeneratorDataset(generator, column_names=["data", "labels"], shuffle=False)
dataloader = dataset.batch(batch_size=32, drop_remainder=True)

# Create the model and loss function
net = RegressionModel()
loss_fn = nn.MSELoss()

# Create an optimizer
optimizer = nn.Adam(net.trainable_params(), learning_rate=0.001)

# Wrap the network with loss function
net_with_loss = nn.WithLossCell(net, loss_fn)

# Create train network
train_network = nn.TrainOneStepCell(net_with_loss, optimizer).set_train()

# Training
epochs = 100
total_steps = num_samples // 32 * epochs

with tqdm(total=total_steps, desc="Training Progress") as pbar:
    for epoch in range(epochs):
        for step, batch in enumerate(dataloader):
            data_batch, label_batch = batch

            loss = train_network(data_batch, label_batch)
            pbar.set_postfix({"loss": loss.asnumpy()})
            pbar.update(1)

        pbar.refresh()

print("Training finished!")

Training Progress:  23%|████████▏                          | 72891/312500 [06:27<21:17, 187.62it/s, loss=7.2506537e-06]

In [142]:
import torch

In [ ]:
torch.